In [1]:
import os
import sys
import numpy as np
import pandas as pd
import tensorflow
import keras

Using TensorFlow backend.


In [2]:
strDataDir = r'/project/hackathon/hackers12/shared'
dfTrainData = pd.read_csv(os.path.join(strDataDir,'DataNormedToTrain','Normed_Training_Data.csv'), index_col=0)
dfTrainData = dfTrainData[dfTrainData.Group != 'Unknown']
dfTestData = pd.read_csv(os.path.join(strDataDir,'DataNormedToTrain','Normed_Test_Data.csv'), index_col=0)
dfTestData = dfTestData[dfTestData.Group != 'Unknown']
dfNameFile = pd.read_excel(os.path.join(strDataDir,'Clean_data.xlsx'), sheet_name = 'Names File')
dfNameFile.sort_values(by=['group_number'], inplace=True)
dfNameFile = dfNameFile[dfNameFile['group_number'] <= 4]
lTrainSamples = dfTrainData.columns[4:]
lTestSamples = dfTestData.columns[4:]

In [3]:
np.array_equal(dfTrainData.Name.values, dfTestData.Name.values)

True

In [4]:
intModelID = 188
epochs = int(53*1.1)
strModel = 'model%s'%intModelID
strModelDir = os.path.join(strDataDir, 'alex_models')
model = keras.models.load_model(os.path.join(strModelDir,strModel+'.h5'))
Xtrain = np.array(dfTrainData[lTrainSamples]).T
Xtest = np.array(dfTestData[lTestSamples]).T

In [5]:
dfTrainData.shape

(388, 82)

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_950 (Dense)            (None, 256)               99584     
_________________________________________________________________
batch_normalization_498 (Bat (None, 256)               1024      
_________________________________________________________________
dense_951 (Dense)            (None, 32)                8224      
_________________________________________________________________
batch_normalization_499 (Bat (None, 32)                128       
_________________________________________________________________
dense_952 (Dense)            (None, 3)                 99        
Total params: 109,059
Trainable params: 108,483
Non-trainable params: 576
_________________________________________________________________


In [7]:
def fOneHot(v):
    if v == 1 or v == 2:
        return [1,0,0]
    if v == 3:
        return [0,1,0]
    if v == 4:
        return [0,0,1]
dfNameFile['OneHot'] = dfNameFile.group_number.apply(lambda x: fOneHot(x))
ytrain = np.array(dfNameFile.set_index('sample_name').loc[lTrainSamples]['OneHot'].values.tolist())
ytest = np.array(dfNameFile.set_index('sample_name').loc[lTestSamples]['OneHot'].values.tolist())

In [8]:
Xtrain.shape

(78, 388)

In [9]:
class_weight = {0:.5,1:1,2:1}
model.fit(Xtrain, ytrain, epochs=epochs, class_weight = class_weight)

Epoch 1/58
78/78 [==============================] - 1s 7ms/step - loss: 1.1061 - acc: 0.3718
Epoch 2/58
78/78 [==============================] - 0s 189us/step - loss: 0.4206 - acc: 0.7821
Epoch 3/58
78/78 [==============================] - 0s 174us/step - loss: 0.2616 - acc: 0.8718
Epoch 4/58
78/78 [==============================] - 0s 183us/step - loss: 0.2023 - acc: 0.9231
Epoch 5/58
78/78 [==============================] - 0s 179us/step - loss: 0.1607 - acc: 0.9615
Epoch 6/58
78/78 [==============================] - 0s 175us/step - loss: 0.1350 - acc: 0.9615
Epoch 7/58
78/78 [==============================] - 0s 169us/step - loss: 0.0954 - acc: 0.9872
Epoch 8/58
78/78 [==============================] - 0s 168us/step - loss: 0.1150 - acc: 0.9744
Epoch 9/58
78/78 [==============================] - 0s 171us/step - loss: 0.0784 - acc: 0.9872
Epoch 10/58
78/78 [==============================] - 0s 174us/step - loss: 0.0739 - acc: 0.9872
Epoch 11/58
78/78 [==============================] 

In [10]:
yPredict = model.predict(Xtest)
yPredictMax = np.argmax(yPredict, axis=1)
ytestMax = np.argmax(ytest, axis=1)

In [11]:
intC = 0
for i,j in zip(yPredictMax, ytestMax):
    if i == j:
        intC+=1

In [12]:
intC/yPredictMax.__len__()

0.7777777777777778

In [13]:
model.save(os.path.join(strDataDir,'LLC_Final_Model.h5'))

In [14]:
ytestMax

array([0, 0, 0, 0, 0, 1, 1, 2, 2])

In [15]:
yPredictMax

array([0, 0, 0, 2, 0, 1, 0, 2, 2])